In [13]:
# !wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
# !wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()  # rstrip removes any trailing white spaces, tabs at end of string
    return genome

In [22]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [3]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [16]:
def bestApproximateMatchEditDistance(p, t):
    """Returns the edit distance between two strings, p and t"""
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)

    # Best Approximate Match Distance is the smallest value of the last row
    return min(D[-1])

In [10]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [23]:
def getkmers(read, kmer_length):
    """our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA."""
    return [read[i:i+kmer_length] for i in range(len(read)+1-kmer_length)]

def overlap_all_pairs(reads, min_length):
    overlap_map = {}
    overlap_graph = {}
    overlap_pairs = []
    
    # use Python dict to associate each k-mer with corresponding set
    suffix_dict = {}
    for read in reads:
        kmers = getkmers(read, min_length)
        
        # for every k-mer in read, add to corresponding set object
        for kmer in kmers:
            if not kmer in suffix_dict.keys():
                # every k-mer will have set object
                suffix_dict[kmer] = set()
            suffix_dict[kmer].add(read)
    
    # for each read a, find all overlaps involving suffix of a
    for read in reads:
        # a's length-k suffix
        suffix = read[-min_length:]
        
        # find all reads containing the suffix from its corresponding set
        matching_reads = suffix_dict[suffix]
        
        # call overlap(a, b, min_length=k) for each read
        for read2 in matching_reads:
            if read2 != read:
                value = overlap(read, read2, min_length)
                if value > 0:
                    overlap_map[(read, read2)] = value
                    overlap_graph[read] = read2
                    overlap_pairs.append((read, read2))
    
    return overlap_pairs, overlap_map, overlap_graph

In [18]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [19]:
# question 1
p = 'GCTGATCGATCGTACG'
bestApproximateMatchEditDistance(p, genome)

3

In [20]:
# question 2
p = 'GATTTACCAGATTGAG'
bestApproximateMatchEditDistance(p, genome)

2

In [25]:
# question 3 and 4
reads, qualities = readFastq('ERR266411_1.for_asm.fastq')
overlap_pairs, overlap_map, overlap_graph = overlap_all_pairs(reads, 30)

print(f'q3: {len(overlap_map)}')
print(f'q4: {len(overlap_graph)}')

q3: 904746
q4: 7161
